In [1]:
import scraper_eod as s
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import time
from scipy.stats import norm
import seaborn as sns
m = s.StockMongo()
import indicator_daily as i

C:\Users\ander\Documents\yahooScraper\scrape\lib\site-packages\cryptography\x509\base.py:531: CryptographyDeprecationWarning: Parsed a negative serial number, which is disallowed by RFC 5280.
  return rust_x509.load_pem_x509_certificate(data)


In [2]:
portfolio = m.get_portfolio(None)

In [3]:
portfolio.count()

C:\Users\ander\AppData\Local\Temp/ipykernel_7580/1202350323.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  portfolio.count()


4

In [4]:
port = pd.DataFrame.from_records(portfolio)

In [5]:
port

,_id,strike_date,rel_risk,high_put_value,low_call_value,high_call_value,low_put_value,highStrikePut,lowStrikeCall,lowStrikePut,highStrikeCall,p,r,ticker,date,value,direction
0,639f90d0271dab5a67b27d79,2023-01-20,0.68200,4.80,11.31,5.80,8.90,87.65,167.7,92.65,172.7,1.088,0.736305,FANG,2022-12-18,10.61,1
1,639f90d0271dab5a67b27d7a,2023-01-20,0.45096,7.20,49.20,21.37,7.00,1600.00,2175.0,1710.00,2300.0,0.408,0.879950,BKNG,2022-12-18,41.80,1
2,639f90d0271dab5a67b27d7b,2023-02-17,0.44000,3.00,4.50,4.30,3.60,105.00,175.0,110.00,180.0,0.408,0.496203,SNOW,2022-12-18,2.10,1
3,639f90d0271dab5a67b27d7c,2023-02-17,0.39200,2.95,3.70,3.95,2.74,155.00,210.0,160.00,215.0,0.408,1.024109,BA,2022-12-18,0.54,1


In [11]:
port.iloc[0]

_id                639f90d0271dab5a67b27d79
strike_date             2023-01-20 00:00:00
rel_risk                              0.682
high_put_value                          4.8
low_call_value                        11.31
high_call_value                         5.8
low_put_value                           8.9
highStrikePut                         87.65
lowStrikeCall                         167.7
lowStrikePut                          92.65
highStrikeCall                        172.7
p                                     1.088
r                                  0.736305
ticker                                 FANG
date                    2022-12-18 00:00:00
value                                 10.61
direction                                 1
Name: 0, dtype: object

In [7]:
formula = list(m.stock_data.options_analisys.find({'ticker': port.iloc[0]['ticker']}))

In [30]:
#apple = m.get_options(port.iloc[0]['ticker'])

symbols = m.stock_data.options_data4.find({'sym': port.iloc[0]['ticker']})
cleanSymbols = [pd.DataFrame.from_records(s['options']) for s in symbols]
op = pd.concat(cleanSymbols)
op.expirationDate = pd.to_datetime(op.expirationDate, format='%Y-%m-%d')
op = op[op['lastTradeDateTime']!='0000-00-00 00:00:00']
op.lastTradeDateTime = pd.to_datetime(op.lastTradeDateTime, format='%Y-%m-%d').dt.date
op.updatedAt = pd.to_datetime(op.updatedAt, format='%Y-%m-%d').dt.date
op.date = op.date - datetime.timedelta(days=1)
#op = op.set_index('date')
o = op

In [22]:
o = apple

In [31]:
#o = o.reset_index()
call1 = o[(o['strike'] == port.iloc[0]['lowStrikeCall'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'expirationDate', 'strike', 'daysBeforeExpiration', 'iv']]
call1.columns = ['date', 'callLow', 'expirationDate', 'strikeCallLow', 'daysBeforeExpiration', 'iv']
call2 = o[(o['strike'] == port.iloc[0]['highStrikeCall'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
call2.columns = ['date', 'callHigh', 'strikeCallHigh']
put1 = o[(o['strike'] == port.iloc[0]['lowStrikePut'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
put1.columns = ['date', 'putLow', 'strikePutLow']
put2 = o[(o['strike'] == port.iloc[0]['highStrikePut'])&(o['expirationDate'] == port.iloc[0]['strike_date'])&(o['type'] == 'CALL')][['date', 'lastPrice', 'strike']]
put2.columns = ['date', 'putHigh', 'strikePutHigh']

condor = call1.merge(call2, on='date', how='inner')
condor = condor.merge(put1, on='date', how='inner')
condor = condor.merge(put2, on='date', how='inner')

condor['condor'] = condor.callLow-condor.callHigh-condor.putHigh+condor.putLow

In [36]:
condor['condor'].tail()

56   -8.49
57   -8.49
58   -8.49
59   -8.49
60   -8.49
Name: condor, dtype: float64